## Database

In [199]:
import pandas as pd
from src.query_funcs import *
import re

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [171]:
artist_details = pd.read_csv("datos/detalles_artistas.csv")
movie_details = pd.read_csv("datos/detalles_peliculas.csv")
artist_movie = pd.read_csv("datos/pelicula_artista.csv")
movies = pd.read_csv("datos/peliculas.csv")

In [172]:
def separar_premios(x):
    try:
        wins = re.search(pattern='\d+\swin', string=x).group(0)
        wins = int(re.sub(" win", "", wins))
    except:
        wins = 0
    try:
        nominations = re.search(pattern='\d+\snomination', string=x).group(0)
        nominations = int(re.sub(" nomination", "", nominations))
    except:
        nominations = 0 
    return (wins, nominations)

def to_minutes(text):
    text = text.replace("h", ":").replace("min", "").replace("m", "")
    splitted = text.split(":")
    try:
        return int(splitted[0])*60+int(splitted[1])
    except:
        return int(splitted[0])
    
def nullify_unknown(x, unknown_str = "Desconocido"):
    if x != unknown_str:
        return x
    else:
        return None

In [173]:
artist_details["premios_descomp"] = artist_details["premios"].apply(separar_premios)

In [174]:
artist_details[["wins", "nominations"]] = artist_details["premios_descomp"].apply(pd.Series).rename(columns={0 : "wins", 1 : "nominations"})

In [175]:
artist_details.drop(columns=["premios", "premios_descomp"], inplace=True)

In [176]:
artist_details["año_nac"] = artist_details["año_nac"].apply(lambda x : nullify_unknown(x, 9999))

In [177]:
artist_details.head()

,actr,año_nac,conocido_por,que_hace,wins,nominations
0,A. Gunaseelan,NaN,'Mandela',"Casting Department, Actress, Casting Director",0,1
1,A. Michelle Harleston,NaN,"'Murder by Numbers', 'The Mark of a Killer', '...",Actor,0,0
2,A.C. Peterson,NaN,"'Narc', 'Shanghai Noon', 'Shooter', 'The Last ...",Actor,0,0
3,A.V.S. Subramanyam,1957.0,"'Roommates', 'Uncle', 'Ori Nee Prema Bangaramk...","Actor, Director, Writer",0,0
4,Øyvind Øvrebø,NaN,"'Tingen, Edderkoppen', 'Demenskoret', 'Kristia...","Actress, Additional Crew",0,0


In [178]:
movies["Mes"] = movies["Mes"].apply(lambda x: nullify_unknown(x, 0))

In [179]:
movies.head()

,Tipo,Titulo,Año,Mes,Id,generos
0,Movie,Las Travesuras de Super Chido,1990,10.0,tt0320656,Action
1,Movie,Velai Kidaichiruchu,1990,8.0,tt0318803,Action
2,Movie,Sor Batalla,1990,8.0,tt0320503,Action
3,Movie,Azaad Desh Ke Gulam,1990,4.0,tt0330019,Action
4,Movie,Suo ming fei dao,1991,11.0,tt0348147,Action


In [180]:
movie_details["direc_imbd"] = movie_details["direc_imbd"].apply(nullify_unknown)
movie_details["guion_imbd"] = movie_details["guion_imbd"].apply(nullify_unknown)
movie_details["argum_imbd"] = movie_details["argum_imbd"].apply(nullify_unknown)
movie_details["durac_imbd"] = movie_details["durac_imbd"].apply(nullify_unknown).apply(lambda x : to_minutes(x) if x else x)

In [181]:
movie_details = movie_details.set_index("id_pelicula").reset_index()

In [286]:
query = '''
CREATE TABLE if not exists artist_info(
        id_artist int primary key,
        nombre varchar(100) not null,
        birth_date numeric,
        conocido_por varchar(300),
        roles varchar(120),
        wins numeric,
        nominations numeric);

CREATE TABLE if not exists genres(
        id_genre int primary key,
        nombre varchar(100) not null
        );
        

CREATE TABLE if not exists movie_details(
        id_movie varchar(100) primary key,
        nombre varchar(100) not null,
        calificacion decimal(10,4),
        director varchar(200),
        guion varchar(200),
        argumento varchar(400),
        minutos numeric
        );

CREATE TABLE if not exists movies(
        id_entry int primary key,
        tipo varchar(50) not null,
        nombre varchar(100) not null,
        anio numeric,
        mes numeric,
        id_movie varchar(100) not null,
        id_genre int not null,
        foreign key (id_movie)
                references movie_details(id_movie)
                on delete restrict
                on update cascade,
        foreign key (id_genre)
                references genres(id_genre)
                on delete restrict
                on update cascade
        );

CREATE TABLE if not exists artist_movies(
        id_entry int primary key,
        id_artist int,
        id_movie varchar(100) not null,
        foreign key (id_artist)
                references artist_info(id_artist)
                on delete restrict
                on update cascade,
        foreign key (id_movie)
                references movie_details(id_movie)
                on delete restrict
                on update cascade) 
        
        '''

conexion = conectar()
query_commit(conexion, query_text=query)

Done!


In [287]:
genre_dict = dict(enumerate(movies["generos"].unique(), start=1))

In [288]:
conexion = conectar()
query_commit_many(conexion, '''insert into genres
                                values (%s,%s)''', tuple(genre_dict.items()))

Done!


In [290]:
conexion = conectar()
query_commit_many(conexion, '''insert into artist_info
                                values (%s,%s,%s,%s,%s,%s,%s)''', [tuple(i) for i in (artist_details.reset_index().values)])

Done!


In [292]:
conexion = conectar()
query_commit_many(conexion, '''insert into movie_details
                                values (%s,%s,%s,%s,%s,%s,%s)''', [tuple(i) for i in (movie_details.values)])

Done!


In [296]:
movies["generos"] = movies["generos"].map({v:k for k,v in genre_dict.items()})

In [298]:
conexion = conectar()
query_commit_many(conexion, '''insert into movies
                                values (%s,%s,%s,%s,%s,%s,%s)''', [tuple(i) for i in (movies.reset_index().values)])

Done!


In [302]:
artist_movie

,Movie ID,Actriz_Actor
0,tt0110660,Tony Ka Fai Leung
1,tt0110363,Luke Edwards
2,tt0110397,Jennifer Barker
3,tt0111045,Miklós Déri
4,tt0111653,Joe Bays
...,...,...
14524,tt27251096,Sin datos
14525,tt27250288,Sin datos
14526,tt27250168,Sin datos
14527,tt27262238,Jacob Schindler


In [300]:
conexion = conectar()
query_commit_many(conexion, '''insert into artist_movies
                                values (%s,%s)''', [tuple(i[1], i[0]) for i in (artist_movie.values)])

InvalidTextRepresentation: la sintaxis de entrada no es válida para tipo integer: «tt0110660»
LINE 2:                                 values ('tt0110660','Tony Ka...
                                                ^
